---
title: keras基础-Dense的输入的秩大于2时的注意事项
date: 2018-08-30 20:17:55
tags: [keras教程, keras基础]
toc: true
xiongzhang: false

---
<span></span>
<!-- more -->


本文代码运行环境:

- windows10
- python3.6
- jupyter notebook
- tensorflow 1.x
- keras 2.x

### 问题描述

keras的Dense层的输入通常是一个二维的矩阵(batch_size, n_input), 但其实, 你的输入的秩可以是大于2的, 比如下面的代码是正确的:

In [9]:
from keras import layers

# 注意keras中, batch size是省略的
input1 = layers.Input((2,3))
dense = layers.Dense(4)
output = dense(input1)
output

<tf.Tensor 'dense_5/add:0' shape=(?, 2, 4) dtype=float32>

而权重的形状是这样的:

In [10]:
dense.kernel

<tf.Variable 'dense_5/kernel:0' shape=(3, 4) dtype=float32_ref>

我们可以看到, 输出`output`的形状是`(batch_size, 2, 4)`, 为什么会是这样的? 我查阅了keras中的文档, 文档中说:`if the input to the layer has a rank greater than 2, then it is flattened prior to the initial dot product with  kernel`, 但是, 实际情况不是这样的, 如果被flatten以后, 输出应该是这样的: 

In [7]:
input1 = layers.Input((2,3))
flat = layers.Flatten()(input1)
dense = layers.Dense(4)
output = dense(flat)
output

<tf.Tensor 'dense_4/BiasAdd:0' shape=(?, 4) dtype=float32>

而权重的形状是这样的:

In [8]:
dense.kernel

<tf.Variable 'dense_4/kernel:0' shape=(6, 4) dtype=float32_ref>

### Dense的算法就是`dot`

很显然, 文档说的有问题, 代码并不是这样的运行的。所以我查阅了Keras的源码, 我发现Dense前向传播的计算方法是在这里:

```python
def call(self, inputs):
    # 理解内部发生了什么, 关键要看这一行
    output = K.dot(inputs, self.kernel)
    if self.use_bias:
        output = K.bias_add(output, self.bias, data_format='channels_last')
    if self.activation is not None:
        output = self.activation(output)
    return output
```

所以, 我们要看一下keras中`dot`的行为:

In [12]:
from keras import backend as K

x = K.placeholder(shape=(2, 2, 3))
y = K.placeholder(shape=(3, 4))
xy = K.dot(x, y)
xy

<tf.Tensor 'Reshape_8:0' shape=(2, 2, 4) dtype=float32>

In [14]:
import tensorflow as tf

sess = tf.InteractiveSession()

In [26]:
import numpy as np
np.random.seed(123)
x_data = np.random.randint(1,10, (2,2,3))
y_data = np.random.randint(1,5, (3,4))
xy_data = xy.eval(feed_dict={x:x_data, y:y_data})
print('x:', x_data)
print('y:', y_data)
print('xy:', xy_data)

x: [[[3 3 7]
  [2 4 7]]

 [[2 1 2]
  [1 1 4]]]
y: [[3 2 1 1]
 [1 1 2 4]
 [4 3 1 4]]
xy: [[[40. 30. 16. 43.]
  [38. 29. 17. 46.]]

 [[15. 11.  6. 14.]
  [20. 15.  7. 21.]]]


这个结果和我们直接用`np.dot`的方法结果是一样的。

In [27]:
np.dot(x_data, y_data)

array([[[40, 30, 16, 43],
        [38, 29, 17, 46]],

       [[15, 11,  6, 14],
        [20, 15,  7, 21]]])

In [29]:
np.dot(x_data, y_data[:, 1])

array([[30, 29],
       [11, 15]])

### 总结

其实两个tensor之间的点乘大家应该懂了, keras官方文档有问题导致对它的理解又产生了疑惑。如果三维的tensor不容易理解的话, 可以去掉batch_size这个维度来看:

In [37]:

x=np.random.randint(1,5,(2, 3))
y=np.random.randint(1,5,(3, 3))
print('正常情况下, x只是个向量:')
np.dot(x[0,:],y)

正常情况下, x只是个向量:


array([28, 26, 16])

In [38]:
print('特殊情况下, x只是个矩阵:')
np.dot(x,y)

特殊情况下, x只是个矩阵:


array([[28, 26, 16],
       [40, 36, 23]])

一句话总结的话, Dense的输入为一个矩阵的时候, 相当于它的每一个行向量分别进入Dense层进行计算, 然后把计算得到的向量拼接成矩阵。有问题来我网站提问:mlln.cn , 转载请注明出处。